In [1]:
from simtk import openmm, unit
import parmed
import mdtraj
import numpy
import math
import logging

from blues.reporters import NetCDF4Reporter, BLUESStateDataReporter
from blues.moves import RandomLigandRotationMove, MoveEngine
from blues.simulation import SystemFactory, SimulationFactory, BLUESSimulation
from blues.integrators import AlchemicalExternalLangevinIntegrator

from blues.reporters import ReporterConfig, init_logger
logger = logging.getLogger(__name__)
logger = init_logger(logger)

# Set a seed/random state
seed = numpy.random.seed(seed=3134)
random_state = numpy.random.RandomState(seed=3134)

ImportError: Could not import openeye-toolkits. SideChainMove class will be unavailable.


/home/nathanlim/anaconda3/envs/blues/lib/python3.6/site-packages/openmmtools/multistate/__init__.py:75: UserWarning: Warning: openmmtools.multistate API is experimental
  warnings.warn('Warning: openmmtools.multistate API is experimental')


In [2]:
# Set Simulation parameters
sim_cfg = { 'platform': 'CPU',
                    'nprop' : 1,
                    'propLambda' : 0.3,
                    'dt' : 1 * unit.femtoseconds,
                    'friction' : 1 / unit.picoseconds,
                    'temperature' : 200 * unit.kelvin,
                    'nIter': 10,
                    'nstepsMD': 20,
                    'nstepsNC': 20,
                    'propSteps': 20,
                    'moveStep' : 10}


totalSteps = int(sim_cfg['nIter'] * sim_cfg['nstepsMD'])
reportInterval = 5
alchemical_atoms = [0,1]
alchemical_functions = {
    'lambda_sterics': 'min(1, (1/0.3)*abs(lambda-0.5))',
    'lambda_electrostatics': 'step(0.2-lambda) - 1/0.2*lambda*step(0.2-lambda) + 1/0.2*(lambda-0.8)*step(lambda-0.8)'
}

In [3]:
# Load a Parmed Structure for the Topology and create our openmm.Simulation
structure = parmed.load_file('../sgill_testsystem/ethlyene_structure.pdb')

# Initialize our move proposal class
rot_move = RandomLigandRotationMove(structure, 'TMP', random_state)
mover = MoveEngine(rot_move)


# Load our OpenMM System and create Integrator
with open('ethylene_system.xml', 'r') as infile:
    xml = infile.read()
    system = openmm.XmlSerializer.deserialize(xml)
integrator = openmm.LangevinIntegrator(sim_cfg['temperature'], sim_cfg['friction'], sim_cfg['dt'])
integrator.setRandomNumberSeed(3134)

alch_integrator = openmm.LangevinIntegrator(sim_cfg['temperature'], sim_cfg['friction'], sim_cfg['dt'])
alch_integrator.setRandomNumberSeed(3134)

alch_system = SystemFactory.generateAlchSystem(system, alchemical_atoms)
ncmc_integrator = AlchemicalExternalLangevinIntegrator(nsteps_neq=sim_cfg['nstepsNC'],
            alchemical_functions=alchemical_functions,   
            splitting="H V R O R V H",
            temperature=sim_cfg['temperature'],
            timestep=sim_cfg['dt'])
ncmc_integrator.setRandomNumberSeed(3134)

In [4]:
# Pack our systems into a single object
systems = SystemFactory(structure, alchemical_atoms)
systems.md = system
systems.alch = alch_system

In [5]:
md_reporters = { 'stream' : {'title' : 'md',
                             'reportInterval': reportInterval,
                             #'totalSteps' : totalSteps,
                             'step' : True,
                             'speed': True,
                             'progress': True,
                             'remainingTime': True,
                             'currentIter' : True,
                             },
                  'traj_netcdf': {'reportInterval': reportInterval}
               }
ncmc_reporters = { 'stream' : {'title' : 'ncmc',
                             'reportInterval': sim_cfg['nstepsNC'],
                             #'totalSteps' : totalSteps,
                             'step' : True,
                             'speed': True,
                             #'progress': True,
                             #'remainingTime': True,
                             'currentIter' : True,
                             'protocolWork' : True,
                             #'alchemicalLambda' : True,
                             }}
md_reporter_cfg = ReporterConfig('bluesv1-ethyleneMD', md_reporters, logger)
md_reporters_list = md_reporter_cfg.makeReporters()
ncmc_reporter_cfg = ReporterConfig('bluesv1-ethyleneNCMC', ncmc_reporters, logger)
ncmc_reporters_list = ncmc_reporter_cfg.makeReporters()
print(md_reporters_list, ncmc_reporters_list)

[<blues.reporters.NetCDF4Reporter object at 0x7f8eecd7c828>, <blues.reporters.BLUESStateDataReporter object at 0x7f8eecd7c748>] [<blues.reporters.BLUESStateDataReporter object at 0x7f8eecd7c908>]


In [6]:
# Pack our simulations into a single object
simulations = SimulationFactory(systems, mover)

simulations.md = SimulationFactory.generateSimFromStruct(structure, system, integrator, 'CPU')
simulations.md = SimulationFactory.attachReporters(simulations.md, md_reporters_list)

simulations.alch = SimulationFactory.generateSimFromStruct(structure, system, alch_integrator, 'CPU')

simulations.ncmc = SimulationFactory.generateSimFromStruct(structure, alch_system, ncmc_integrator, 'CPU')
simulations.ncmc = SimulationFactory.attachReporters(simulations.ncmc, ncmc_reporters_list)

In [7]:
blues = BLUESSimulation(simulations, sim_cfg)
blues.run()

#"Iter"	"Step"	"protocolWork"	"Speed (ns/day)"
ncmc: 0	20	1.0440742091850204	0
#"Iter"	"Step"	"Speed (ns/day)"
md: 0	5	0
md: 0	10	123
md: 0	15	87.8
md: 0	20	107
ncmc: 1	20	0.4219119985596887	45
md: 1	25	47.7
md: 1	30	51.7
md: 1	35	56.4
md: 1	40	57.7
ncmc: 2	20	-0.5186188538200988	44.1
md: 2	45	44.7
md: 2	50	44.1
md: 2	55	46.7
md: 2	60	49.5
ncmc: 3	20	-0.06514108337480284	44.8
md: 3	65	46.5
md: 3	70	49.4
md: 3	75	51.8
md: 3	80	54.1
ncmc: 4	20	0.16508255171697056	49.2
md: 4	85	50.5
md: 4	90	51.2
md: 4	95	52.5
md: 4	100	54.6
ncmc: 5	20	-0.780839780238944	50.2
md: 5	105	51.6
md: 5	110	53.1
md: 5	115	55
md: 5	120	56.5
ncmc: 6	20	-0.0025249860902839764	53
md: 6	125	52.3
md: 6	130	53.9
md: 6	135	55.5
md: 6	140	57.1
ncmc: 7	20	-1.1606892547842782	54
md: 7	145	55.1
md: 7	150	56.3
md: 7	155	56.4
md: 7	160	57.4
ncmc: 8	20	0.8770331401067734	53.1
md: 8	165	53.1
md: 8	170	54.1
md: 8	175	55.3
md: 8	180	56.3
ncmc: 9	20	1.6508201749895417	53.2
md: 9	185	52.9
md: 9	190	54
md: 9	195	55.1
md: 9	200	56.1
